# **RAZOR GROUP CASE STUDY - PYTHON**

The dataset provided contains information about sellers. 
The task is to sanitize and analyze the dataset, and then apply selection criteria to the dataset to filter out the sellers best suited for the Razor team to approach and acquire.

### Steps taken:
#### Cleaning the data:

* As razor is not presently acquiring Chinese sellers, the country codes were parsed out and all results from China were dropped.
* The number of products the sellers carry was parsed out.
* The number and percentage of positive ratings were parsed out.
* The amount of time for which the ratings were displayed was parsed out.
* The business name was parsed out.

#### Analyzing the data:

* The date and category columns were checked to see if they might be needed for the analysis, but they were the same for all rows.
* The datatypes were changed to the appropriate type.
* The data was filtered by product count.
* The resulting data was filtered by the percentage of positive ratings.
* The resulting data filtered by sorting by the ratio of the number of positive ratings to the product count.
* The data resulting from that was filtered by the sum of the hero product ratings.
* The sellers for whom the percentage of negative reviews increased continuously were discarded from the dataset.

#### Final cleanup and result extraction:

* The email ids were parsed out, wherever they existed.
* The phone numbers were parsed out wherever they existed.
* A [table](#details) containing the sellers best suited for acquiring and some useful metrics was created.
* A [table](#contacts) containing the contact details of the sellers was created.



## Cleaning the data

#### Importing pandas and creating the dataframes:

In [1]:
import pandas 

import warnings
warnings.filterwarnings('ignore')

In [2]:
# This reads the csv and save it to the dataframe named file
file = pandas.read_csv('Sample_Longlist_Data.csv')

# cfl is the cleaned data frame
cfl = pandas.DataFrame()

#### Parsing for the country code:

In [3]:
#Taking the last 2 characters of the business address
file['countrycode'] = file['businessaddress'].str[-2:]

#Dropping rows for businesses based in China
for i in range(1839):
     if file['countrycode'][i] == 'CN':
         file.drop(i, axis = 0, inplace = True)

#resetting the indices
file.index = range(len(file))
    
#file.iloc[:]

#### Parsing for the count of products:

This check makes sure that the product count is formatted in the same way for every row in the product counts:

In [4]:
# This returns the number of unique results when we take the last seven characters of the sellerproductcount column
print(len(pandas.unique(file['sellerproductcount'].str[-8:])))

2


As the code also counts null values, we now know that every non-null row is formatted such that the last eight characters are the same. The last 8 characters are  ' results'.

In [5]:
# We create a column for the productcount in the cfl dataframe by removing the last 8 characters from the sellerproductcount column of file.
cfl['Product_count'] = file['sellerproductcount'].str[:-8] 

# We split each row using a space as the delimiter
cfl['Product_count'] = cfl['Product_count'].str.split(' ')

# We select the last string 
cfl['Product_count'] = cfl['Product_count'].str[-1]


#### Parsing for the number and percentage of ratings: 

In [19]:
#This finds the percentage of positive ratings by splitting by '%' and taking the string to the left of it.
cfl['%_of_+ve_ratings'] = file['sellerratings'].str.split('%').str[0]

#This finds the number of positive ratings by splitting by '(' and taking the string to the left of it.
cfl['Num_of_+ve_ratings'] = file['sellerratings'].str.split('(').str[1].str.split(' ').str[0]

#This finds time for which the data has been collected 
cfl['Rating_since'] = file['sellerratings'].str.split('(').str[0].str.split('months').str[0].str.split(' ').str[-2]

# This replaces all rows that say 12 with 12 months, to specify the duration more clearly
for i in range(len(cfl)):
    if cfl['Rating_since'][i] == '12':
        cfl['Rating_since'][i] = '12 months'
cfl.iloc[:]    

,Product_count,%_of_+ve_ratings,Num_of_+ve_ratings,Rating_since,Business_name,Seller_details,Business_address,Num_brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings
0,100000.0,88,118,12 months,Lohas Living Inc,Lohas Living Inc James Mazzello US 845 3RD Ave...,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,16,17,11,12,5873,4957
1,0.0,90,566,12 months,1a-Handelsagentur,Herzlich Willkommen im Amazon Shop von 1a-Hand...,1a HandelsagenturLindenallee 2MalchowMecklenbu...,0,2,2,9,0,0
2,28.0,NaN,NaN,NaN,NaN,7830jeff is committed to providing each custom...,NaN,13,0,0,0,3284,2944
3,919.0,97,116,12 months,Oliver Mills,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,Kaiserstr.12Borgholzhausen33829DE,15,0,0,3,1446,921
4,0.0,76,3721,12 months,ABC-Schnäppchenmarkt GmbH,Impressum: ABC-Schnäppchenmarkt GmbH \n Jösser...,Jösser Weg10Petershagen32469DE,0,18,20,20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,52.0,100,16,12 months,AD-ACCESS-Zündholz Riesa GmbH,Zündholz Riesa is committed to providing each ...,Paul-Greifzu-Straße22Riesa01591DE,8,0,0,0,65,41
598,177.0,100,10,12 months,15289413,Impressum:\n\nStephanie Severt\n\nZUR ROSA KUH...,NaN,15,0,0,0,91,18
599,1000.0,91,3075,12 months,Reinhard Joormann -FansandTrends- e.K.,Reinhard Joormann -FansandTrends- e.K. Im St...,Im Stadtsfeld 55Dorsten46282DE,15,6,8,7,636,466
600,2000.0,100,4,lifetime,Zwoofer.com,Zwoofershop is committed to providing each cus...,Orteliusstraat 287-3AmsterdamNoord-Holland1056...,16,0,0,0,27269,11333


#### Parsing for the business name and adding the rest of the columns:

In [7]:
cfl['Business_name'] = file['seller business name'].str[14:]
cfl['Seller_details'] = file['sellerdetails']
cfl['Business_address'] = file['businessaddress']
cfl['Num_brands'] = file['Count of seller brands']
cfl['Max % of negative seller ratings - last 30 days'] = file['Max % of negative seller ratings - last 30 days']
cfl['Max % of negative seller ratings - last 90 days'] = file['Max % of negative seller ratings - last 90 days']
cfl['Max % of negative seller ratings - last 12 months'] = file['Max % of negative seller ratings - last 12 months']
cfl['Hero Product 1 #ratings'] = file['Hero Product 1 #ratings']
cfl['Hero Product 2 #ratings'] = file['Hero Product 2 #ratings']


# 
# Analyzing the data

In [8]:
# To find the number of unique dates and categories
print(len(pandas.unique(file['Date Added'])))
print(len(pandas.unique(file['category'])))

1
1


The date and category are the same for all rows.
The date was 15-Nov-20, and the category was Garden

#### Checking the datatypes and changing the required columns' datatype:

In [9]:
#This gives you the data types of the values in  your columns
cfl.dtypes

Product_count                                        object
%_of_+ve_ratings                                     object
Num_of_+ve_ratings                                   object
Rating_since                                         object
Business_name                                        object
Seller_details                                       object
Business_address                                     object
Num_brands                                            int64
Max % of negative seller ratings - last 30 days       int64
Max % of negative seller ratings - last 90 days       int64
Max % of negative seller ratings - last 12 months     int64
Hero Product 1 #ratings                               int64
Hero Product 2 #ratings                               int64
dtype: object

When changing the datatype into a float, if there are any commas in the items, they get converted to decimals. Which is why we first remove all commas from the Product_count column. We then change the datatype of the coolumns containig product counts, percentage of positive ratings, and number of positive ratings to a float.

In [10]:
cfl['Product_count'] = cfl['Product_count'].str.replace(',','')
cfl['Product_count'] = pandas.to_numeric(cfl['Product_count'],errors='coerce')
cfl['%_of_+ve_ratings'] = pandas.to_numeric(cfl['%_of_+ve_ratings'],errors='coerce')
cfl['Num_of_+ve_ratings'] = pandas.to_numeric(cfl['Num_of_+ve_ratings'],errors='coerce')
cfl.dtypes

Product_count                                        float64
%_of_+ve_ratings                                     float64
Num_of_+ve_ratings                                   float64
Rating_since                                          object
Business_name                                         object
Seller_details                                        object
Business_address                                      object
Num_brands                                             int64
Max % of negative seller ratings - last 30 days        int64
Max % of negative seller ratings - last 90 days        int64
Max % of negative seller ratings - last 12 months      int64
Hero Product 1 #ratings                                int64
Hero Product 2 #ratings                                int64
dtype: object

#### Sorting and filtering by the product count:

In [11]:
#This transforms all the null values to 0
cfl['Product_count'] = cfl['Product_count'].fillna(0)
#This copies the data into a new dataframe and sorts by the first column
cfl4=cfl.sort_values(by=['Product_count'], ascending = False,inplace=False)

#This finds the average product count excluding extremely large counts
k=0
l=0
for i in range(602):
    if cfl4['Product_count'][i] != 0.0 and cfl4['Product_count'][i]<100000:
        k+=cfl4['Product_count'][i]
        l+=1

mean = k/l

print(mean)

4544.1334745762715


In [12]:
#This gets rid of all the columns that have a product count below the average
for i in range(len(cfl4)):
    if cfl4['Product_count'][i]<mean:
        cfl4.drop(i, axis = 0, inplace = True)
        
cfl4.index = range(len(cfl4))

#cfl4.iloc[:] #--> This lets you display the data frame that has been obtained
        

#### Sorting by percentage of positive ratings

In [13]:
#All the null values are replaced by 0
cfl4['%_of_+ve_ratings'] = cfl4['%_of_+ve_ratings'].fillna(0)

# The values are sorted by the percentage of positive ratings
cfl4=cfl4.sort_values(by=['%_of_+ve_ratings'], ascending = False,inplace=False)

# All sellers with ratings below 90% are removed 
for i in range(len(cfl4)):
    if cfl4['%_of_+ve_ratings'][i]<=90:
        cfl4.drop(i, axis = 0, inplace = True)
        
#The indices are reset        
cfl4.index = range(len(cfl4))

#cfl4.iloc[:] #--> This lets you display the data frame that has been obtained

When the number of products is higher, there generally are a higher number of customers and consequently, a higher number of ratings. So the ratio between the number of positive ratings and the number of products is taken as a metric.

In [14]:
# This creates a new empty column 
cfl4['Ratio']=""

# This places the ratios of the percentage of positive ratings to the number of products into the 'Ratio' column 
for i in range(int(len(cfl4))):
    cfl4['Ratio'][i] = (cfl4['Num_of_+ve_ratings'][i])/(cfl4['Product_count'][i])

# This sorts the data by the ratio
cfl4=cfl4.sort_values(by=['Ratio'], ascending = False,inplace=False)
cfl4.index = range(len(cfl4))

# Filters out the bottom half
for i in range(int(len(cfl4))):
    if i>len(cfl4)/2:
        cfl4.drop(i, axis = 0, inplace = True)

# Resets the indices
cfl4.index = range(len(cfl4))
    
#cfl4.iloc[:] #--> This lets you display the data frame that has been obtained

In [15]:
# Creates a coulmn containing the sum of the hero product ratings
cfl4['Sum_hero'] = cfl4['Hero Product 1 #ratings'] + cfl4['Hero Product 2 #ratings']

# Sorts by the sum of hero product ratings
cfl4 = cfl4=cfl4.sort_values(by=['Sum_hero'], ascending = False,inplace=False)
# Resets the indices
cfl4.index = range(len(cfl4))

# Selects the top 20
for i in range(len(cfl4)):
    if i>=20:
        cfl4.drop(i, axis = 0, inplace = True)

# Resets indices
cfl4.index = range(len(cfl4))
        
#cfl4.iloc[:] #--> This lets you display the data frame that has been obtained

In [16]:
# Deselects all the sellers whose negative ratings increased continuously over time
for i in range(len(cfl4)):
    if cfl4['Max % of negative seller ratings - last 30 days'][i]>cfl4['Max % of negative seller ratings - last 90 days'][i] and cfl4['Max % of negative seller ratings - last 90 days'][i]>cfl4['Max % of negative seller ratings - last 12 months'][i]:
        cfl4.drop(i,axis=0,inplace=True)
        
# Resets indices
cfl4.index = range(len(cfl4))

#cfl4.iloc[:] #--> This lets you display the data frame that has been obtained

## Final cleanup and display

#### Parsing for the email and phone number:

In [17]:
#This returns the listed email addresses as a list
cfl4['Seller_email'] = cfl4['Seller_details'].str.replace(':',': ').str.replace(')',') ').str.findall('(\S+@\S+)')

#This fills the column with data from the seller details, and replaces all \n with spaces. 
cfl4['Seller_PhNum'] = cfl4['Seller_details'].str.replace('\n',' ')

for i in range(len(cfl4)):
    #This creates a list of all the characters that are allowed in phone numbers
    numchars=['.',',','-',' ','/','(',')','0','1','2','3','4','5','6','7','8','9']
    

# The phone numbers are always preceded by 'Tel:','tel:','Telefon:','telefon:', or '+'.
# In each of those cases, we split the string using the above substrings as delimiters, and then take the string to the right of it.
# We then start checking all the characters from the left, and stop when a character that is not present in the allowed chracters is reached.
# We then use the string that is obtained to get the phone numbers.

    if '+' in str(cfl4['Seller_PhNum'][i]):
        cfl4['Seller_PhNum'][i] = cfl4['Seller_PhNum'][i].split('+')[1]
        x=0
        for a in range(len(cfl4['Seller_PhNum'][i])):
            if (cfl4['Seller_PhNum'][i])[a] in numchars:
                x+=1
            else:
                break
        cfl4['Seller_PhNum'][i] = (cfl4['Seller_PhNum'][i])[:x]
                
    
    elif 'Tel.:' in str(cfl4['Seller_PhNum'][i]):
        cfl4['Seller_PhNum'][i] = cfl4['Seller_PhNum'][i].split('Tel.:')[1]
        x=0
        for a in range(len(cfl4['Seller_PhNum'][i])):
            if (cfl4['Seller_PhNum'][i])[a] in numchars:
                x+=1
            else:
                break
        cfl4['Seller_PhNum'][i] = (cfl4['Seller_PhNum'][i])[:x]
        
    
    elif 'Telefon:' in str(cfl4['Seller_PhNum'][i]):
        cfl4['Seller_PhNum'][i] = cfl4['Seller_PhNum'][i].split('Telefon:')[1]
        x=0
        for a in range(len(cfl4['Seller_PhNum'][i])):
            if (cfl4['Seller_PhNum'][i])[a] in numchars:
                x+=1
            else:
                break
        cfl4['Seller_PhNum'][i] = (cfl4['Seller_PhNum'][i])[:x]
        
    
    elif 'tel.:' in str(cfl4['Seller_PhNum'][i]):
        cfl4['Seller_PhNum'][i] = cfl4['Seller_PhNum'][i].split('tel.:')[1]
        x=0
        for a in range(len(cfl4['Seller_PhNum'][i])):
            if (cfl4['Seller_PhNum'][i])[a] in numchars:
                x+=1
            else:
                break
        cfl4['Seller_PhNum'][i] = (cfl4['Seller_PhNum'][i])[:x]
        
    
    elif 'telefon:' in str(cfl4['Seller_PhNum'][i]):
        cfl4['Seller_PhNum'][i] = cfl4['Seller_PhNum'][i].split('telefon:')[1]
        x=0
        for a in range(len(cfl['Seller_PhNum'][i])):
            if (cfl4['Seller_PhNum'][i])[a] in numchars:
                x+=1
            else:
                break
        cfl4['Seller_PhNum'][i] = (cfl4['Seller_PhNum'][i])[:x]
        
    
    else:
         cfl4['Seller_PhNum'][i] = 'Not Available'
            
# This sorts by the number of products
cfl4=cfl4.sort_values(by=['Product_count'], ascending = False,inplace=False) 
# This resets the indices
cfl4.index = range(len(cfl4))

#cfl4.iloc[:] #--> This lets you display the data frame that has been obtained

<a id='details'></a>
#### Sellers and useful values:

In [18]:
#Creating a dataframe to store the final list of sellers and some key values
finallist = cfl4.loc[:,['Business_name','Product_count','%_of_+ve_ratings','Num_of_+ve_ratings','Sum_hero','Num_brands']]

finallist.iloc[:]

,Business_name,Product_count,%_of_+ve_ratings,Num_of_+ve_ratings,Sum_hero,Num_brands
0,Zoreno Versandhandel GmbH,100000.0,98.0,2630.0,30781,16
1,itenga GmbH,50000.0,91.0,5150.0,59593,15
2,Milovan Stojkovic,30000.0,92.0,699.0,130005,16
3,TransPal GmbH,20000.0,93.0,3279.0,19519,16
4,VIP Plaza Japan Inc,10000.0,92.0,159.0,124572,16
5,Lorde Mall s.r.o.,10000.0,93.0,203.0,22670,15
6,Autoteile Jakobs GmbH,10000.0,94.0,305.0,17080,15
7,Geschenke Direkt GmbH,9000.0,99.0,496.0,13444,16
8,VARIA System GmbH,7000.0,94.0,196.0,118153,16
9,Kessler electronic GmbH,7000.0,97.0,3416.0,79757,16


<a id='contacts'></a>
#### Sellers and contact details if available:

In [19]:
sellercontact=cfl4.loc[:,['Business_name','Seller_email','Seller_PhNum']]

sellercontact.iloc[:]

,Business_name,Seller_email,Seller_PhNum
0,Zoreno Versandhandel GmbH,[ade.support@zoreno.com],Not Available
1,itenga GmbH,[info@itenga.de],Not Available
2,Milovan Stojkovic,[info@limuno.com],49 59 71/ 8 00 38 03
3,TransPal GmbH,[support@yovivo.de],49 30 436 07 330
4,VIP Plaza Japan Inc,[],Not Available
5,Lorde Mall s.r.o.,[],Not Available
6,Autoteile Jakobs GmbH,"[info@lenkgetriebe.netGeschäftsführung:, dn@pr...",06831-9458970
7,Geschenke Direkt GmbH,[],0314 / 25 41 71 50
8,VARIA System GmbH,[shop@varia-store.com],0371 400 7682
9,Kessler electronic GmbH,[info@kessler-electronic.de],49 (0)2273 / 991-9325 (
